In [1]:
from src.dl_data_pipeline.validator import TypeValidator, ShapeValidator, MinMaxValidator, MeanVarValidator
from src.dl_data_pipeline.pipeline.pipe_node import PipelineNode
from src.dl_data_pipeline.pipeline.data_pipeline import Pipeline
from src.dl_data_pipeline import deferred_execution, InputNode
from src.dl_data_pipeline.process_functions import (any_process, process_2d)
import numpy as np

@deferred_execution
def print_infos(x):
    print(x.shape)
    return x

input1 = PipelineNode(name = "Input1")
x, y = any_process.rescale(input1, 0, 1)
x = process_2d.resize_with_max_distortion(x, (256, 256), 0.5)
x = print_infos(x)
x = process_2d.padding_2d(x, (256, 256), 0)

pipe = Pipeline([input1], x)
pipe.add_validator(TypeValidator(np.ndarray), 0)
pipe.add_validator(ShapeValidator((256, 256, 3)), 0)

import matplotlib.pyplot as plt
plt.imshow(pipe(np.random.rand(1920, 1080, 3)))

ValueError: too many values to unpack (expected 2)

In [1]:
from src.dl_data_pipeline.pipeline import PipelineNode, Pipeline
a = PipelineNode()
x = a[0]
y = a[1]
pipe = Pipeline(a, [x, y])

In [2]:
pipe((0,1))

[0, 1]

In [11]:
class Test:
    def __getitem__(self, key):
        print(key)

a = Test()


(slice(1, 2, 1), 2, 123)
